# Практика
Используемые библиотеки

In [4]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [6]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
# s = requests.get(url).content
# df=pd.read_csv(io.StringIO(s.decode('UTF8')))
### Если не работает загрузка on-line
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [7]:
# Ваше решение
def analis_data_preparing(x):
    '''Функция для подготовки результатов анализов'''
    try:
        return float(x)
    except ValueError:
        return 0
    
def ne_bolee(x):
    '''Проверяет нахождение строки "не более"'''
    if 'не более' in x:
        return True
    else:
        return False

def v_predelah(x):
    '''Проверяет нахождение строки "в пределах"'''
    if 'в пределах' in x:
        return True
    else:
        return False    
    
def ne_nizhe_normy(df):
    '''Вспомогательная функция для сравнения нормативов'''
    x = df['Результат анализа'] >= df['Значения нормативов. Минимум']
    return x

def ne_vyshe_normy(df):
    '''Вспомогательная функция для сравнения нормативов'''
    y = df['Результат анализа'] <= df['Значения нормативов. Максимум']
    return y

def v_predelah_ok(df):
    '''Анализ для кейса "В пределах"'''
    x = [ne_vyshe_normy(df), ne_nizhe_normy(df)]
    y = x[0] #сразу зададим тип
    for cnt in range(len(x[0])):
        if x[0][cnt] and x[1][cnt]:
           y[cnt] = 'в норме'
        elif x[0][cnt]:
           y[cnt] = 'ниже нормы'
        else:
            y[cnt] = 'выше нормы'
    return y
    
# Разделяем интервальные значения нормативов
df['Значения нормативов. Максимум'] = (list(map(lambda x: float(x.split()[2].replace(',', '.').split('-')[-1]), 
                                                df['Норматив'])))
df['Значения нормативов. Минимум'] = (list(map(lambda x: float(x.split()[2].replace(',', '.').split('-')[0]), 
                                               df['Норматив'])))

# Привели результаты к float
df['Результат анализа'] = df['Результат анализа'].apply(analis_data_preparing) 

# Если условие "Не более" то просто сравниваем и выдаем значение
df.loc[df['Норматив'].apply(ne_bolee),'Итог'] = list(map(lambda x: 'в норме' if x else 'выше нормы', 
                                                           ((df[df['Норматив'].apply(ne_bolee)]['Результат анализа'] 
                                          <= df[df['Норматив'].apply(ne_bolee)]['Значения нормативов. Максимум']))))
#Если в пределах, то идем по более сложной логике, чтобы понять, выше или ниже предела
df.loc[df['Норматив'].apply(v_predelah),'Итог'] = v_predelah_ok(df)

#Наводим красоту
df.drop(['Значения нормативов. Максимум','Значения нормативов. Минимум'], axis=1, inplace=True)
df.set_index('Показатель', inplace=True)
df

,Единица измерений,Результат анализа,Норматив,Итог
Показатель,,,,
pH,единицы pH,8.400,в пределах 6-9,в норме
Запах,баллы,1.000,не более 2-3,в норме
Цветность,градусы,0.000,не более 30,в норме
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,в норме
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",в норме
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",в норме
Нитраты (по NO3),мг/дм3,24.000,не более 45,в норме
Фосфаты (P),мг/дм3,0.360,"не более 3,5",в норме
Хлориды (Cl),мг/дм3,200.000,не более 350,в норме


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [58]:
# Ваше решение
box = ['orange']*5 + ['apple']*4
n = 1000000
oranges = 0
cnt = 0
while cnt < n:
    x = (np.random.choice(box, 3, replace=False))
    if x[0] == x[1] == x[2] == 'orange':
        oranges += 1
    cnt +=1
chance = oranges/cnt
chance

0.118853

#### Результат подтвержден экспериментально

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [115]:
# Ваше решение
details = ['standart']*7 + ['nestandart']*3
our_cases = 0
cnt = 0
n = 1000000
while cnt < n:
    cnt += 1
    case = np.random.choice(details, 2, replace = False)
    if case[0] == 'nestandart' and case[1] == 'standart':
        our_cases += 1
print(our_cases/cnt)

0.233417


#### Результат подтвержден экспериментально